# Import

In [1]:
import pandas as pd
import numpy as np

# Constants

In [61]:
VOCAB_SIZE = 2500

TRAINING_DATA_FILE = 'Training/train-data.txt'
TEST_DATA_FILE = 'Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'Testing/test-features.txt'
TEST_TARGET_FILE = 'Testing/test-target.txt'

# Read & Load Fetures from .txt Files into NumPy Array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [5]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [6]:
sparse_train_data[-5:]

array([[5795, 2026,    0,    2],
       [5795, 2033,    0,    1],
       [5795, 2273,    0,    1],
       [5795, 2325,    0,    1],
       [5795, 2366,    0,    1]])

In [7]:
print('Nr of rows in training file:', sparse_train_data.shape[0])
print('Nr of rows in test file:', sparse_test_data.shape[0])

Nr of rows in training file: 258380
Nr of rows in test file: 117789


In [8]:
print('Nr of unique emails in training file:', np.unique(sparse_train_data[:, 0]).size)
print('Nr of unique emails in testing file:', np.unique(sparse_test_data[:, 0]).size)

Nr of unique emails in training file: 4013
Nr of unique emails in testing file: 1724


## Creating Empty DataFrame

In [9]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [10]:
len(column_names)

2502

In [11]:
index_names = np.unique(sparse_train_data[:, 0])

In [12]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

In [13]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create a Full Matrix from a Sparse Matrix

In [14]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, frequency_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a Pandas DataFrame.
    
    sparse_matrix -- numpy array
    nr_words -- size of the vocabluary. Total number of tokens.
    doc_idx -- pozition of the document id in the sparse matrix. Default: 1st column.
    word_idx -- pozition of the word id in the sparse matrix. Default: 2nd column.
    cat_idx -- pozition of the label (spam is 1, nonspam is 0). Default: 3rd column.
    frequency_idx -- pozition of occurence of word in the sparse matrix. Default: 4th column.
    """
    
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurence = sparse_matrix[i][frequency_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix

In [15]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 7.38 s


In [16]:
 full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5789,0,3,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,3,1,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5794,0,1,1,1,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
5795,0,3,4,2,0,5,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model

## Calculating the Probability of Spam

In [17]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probability of Spam is:', prob_spam)

Probability of Spam is: 0.310989284824321


## The Total Number of Words / Tokens

In [18]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,1,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
email_length = full_train_features.sum(axis=1)
email_length.shape

(4013,)

In [20]:
email_length[: 5]

DOC_ID
0     87
1     53
2     40
3    183
4     43
dtype: int64

In [21]:
total_wc = email_length.sum()
total_wc

429240

## Number of Tokens in Spam & Ham Emails

In [23]:
spam_length = email_length[full_train_data.CATEGORY == 1]
spam_length.shape

(1248,)

In [27]:
spam_wc = spam_length.sum()
spam_wc

176335

In [31]:
ham_length = email_length[full_train_data.CATEGORY == 0]
nonspam_wc = ham_length.sum()

In [32]:
nonspam_wc + spam_wc - total_wc == 0

True

In [36]:
print('Avg nr of words in spam email {:.0f}'.format(spam_wc / spam_length.shape[0]))

Avg nr of words in spam email 141


In [37]:
print('Avg nr of words in nonspam email {:.0f}'.format(nonspam_wc / ham_length.shape[0]))

Avg nr of words in nonspam email 91


## Summing the Tokens Occuring in Spam

In [38]:
full_train_features.shape

(4013, 2500)

In [39]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]

In [40]:
train_spam_tokens.shape

(1248, 2500)

In [42]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1 #Laplace Smoothing
summed_spam_tokens.shape

(2500,)

In [44]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
train_ham_tokens.shape

(2765, 2500)

In [47]:
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1 #Laplace Smoothing
summed_ham_tokens.shape

(2500,)

In [50]:
summed_ham_tokens.tail()

2495    15
2496    29
2497    16
2498     6
2499    33
dtype: int64

In [49]:
train_ham_tokens[2499].sum() + 1

33

## P(Token | Spam) - Probability that a Token Occurs given the Email is Spam

In [51]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.012184
1    0.005228
2    0.006805
3    0.011307
4    0.006816
dtype: float64

In [52]:
prob_tokens_spam.sum()

1.0

## P(Token | Ham) - Probability that a Token Occurs given the Email is Nonspam

In [54]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)
prob_tokens_nonspam[:5]

0    0.021472
1    0.010141
2    0.008007
3    0.003673
4    0.006312
dtype: float64

In [55]:
prob_tokens_nonspam.sum()

1.0

# P(Token) - Probability that Token Occurs

In [57]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc

In [58]:
prob_tokens_all.sum()

1.0

# Save the Trained Model

In [60]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test Data

In [62]:
%%time
full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

Wall time: 3.49 s


In [63]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [64]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)